In [5]:
import pandas as pd
import numpy as np

daily = pd.read_csv("../data/clean_pm25.csv", parse_dates=["datetime"], index_col="datetime")
df_all = daily.reset_index().rename(columns={"datetime": "ds", "pm25": "y"})
H = 60
df_train = df_all.iloc[:-H].copy()
df_test  = df_all.iloc[-H:].copy()

df_train.head(), df_test.shape


(          ds           y
 0 2010-01-01  129.000000
 1 2010-01-02  144.333333
 2 2010-01-03   78.375000
 3 2010-01-04   29.291667
 4 2010-01-05   43.541667,
 (60, 2))

In [7]:
import sys, pkgutil, importlib, inspect

# 1) Where is "prophet" coming from?
import prophet
print("prophet version:", getattr(prophet, "__version__", "unknown"))
print("prophet module path:", prophet.__file__)

# 2) What is Prophet?
from prophet import Prophet
print("Prophet object:", Prophet)

# 3) Sanity: try constructing a bare model (should NOT error)
try:
    _m = Prophet()
    print("Prophet() constructed OK.")
except Exception as e:
    print("Prophet() construction failed:", repr(e))


prophet version: 1.1.5
prophet module path: c:\Users\sergi_nyo0upt\OneDrive\Documents\Projects\pollution-forecasting\.venv\Lib\site-packages\prophet\__init__.py
Prophet object: <class 'prophet.forecaster.Prophet'>
Prophet() construction failed: AttributeError("'Prophet' object has no attribute 'stan_backend'")


In [6]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

def mape(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    mask = y_true != 0
    return (np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]).mean()) * 100

m_base = Prophet(
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    changepoint_prior_scale=0.2,
)
m_base.fit(df_train[["ds","y"]])

fcst_test = m_base.predict(df_test[["ds"]])
pred = fcst_test["yhat"].to_numpy()

metrics_prophet = {
    "MAE":  mean_absolute_error(df_test["y"], pred),
    "RMSE": mean_squared_error(df_test["y"], pred, squared=False),
    "MAPE": mape(df_test["y"], pred),
}
metrics_prophet


AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [ ]:
import matplotlib.pyplot as plt

# Test window plot
plt.figure(figsize=(12,5))
plt.plot(df_test["ds"], df_test["y"], label="Test (actual)", alpha=0.8)
plt.plot(df_test["ds"], pred, label="Prophet", linewidth=2)
plt.title("Prophet — test window")
plt.legend(); plt.show()

# Future forecast H days ahead
future_all = m_base.make_future_dataframe(periods=H, freq="D")
fcst_all = m_base.predict(future_all)
future = fcst_all.tail(H).set_index("ds")[["yhat","yhat_lower","yhat_upper"]]

plt.figure(figsize=(12,5))
plt.plot(df_all["ds"], df_all["y"], label="History", alpha=0.6)
plt.plot(future.index, future["yhat"], label=f"Prophet future {H}d", linewidth=2)
plt.fill_between(future.index, future["yhat_lower"], future["yhat_upper"], alpha=0.2)
plt.xlim(df_all["ds"].iloc[-180], future.index[-1])
plt.title("Prophet future forecast (zoomed on end)")
plt.legend(); plt.show()

# Save for the app
future[["yhat"]].rename(columns={"yhat":"forecast"}).to_csv("../data/forecast_prophet.csv")
future.head()
